## Etherscan Notebook 

Looking at how to derive price for tokens and gas fees by pool.  Pool example can be found [here](https://app.uniswap.org/explore/pools/ethereum/0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640)

In [1]:
import requests
import time 

import pandas as pd
import numpy as np
from web3 import Web3


In [2]:
ETHERSCAN_API_KEY = '16FCD3FTVWC3KDK17WS5PTWRQX1E2WEYV2'
ETHERSCAN_URL = "https://api.etherscan.io/api"
POOL0_ADDRESS="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640" # USDC / WETH (0.05%) 
POOL0_TXN_FEE = 0.0005
POOL1_ADDRESS="0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8" # USDC / WETH (0.3%)
POOl1_TXN_FEE = 0.003

GWEI_SCALER = 1e9

In [3]:
def fetch_latest_block(url, api_key):
    # Parameters for the API request
    params = {
        "module": "proxy",
        "action": "eth_blockNumber",
        "apikey": api_key
    }
    
    # Make the request
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        #st.error(f"API request failed with status code {response.status_code}")
        (f"API request failed with status code {response.status_code}") 
        return None
    
    # Handle the response
    if response.status_code == 200:
        result = response.json()
        if result.get("status") != "0" and "result" in result:
            latest_block = int(result['result'], 16)  # Convert hex to int
            #print(f"Latest Block Number: {latest_block}")
            return latest_block
        else:
            print(f"Unexpected response or error: {result.get('message', 'Unknown error')}")
            return None
    else:
        print(f"Failed to fetch the latest block. Status code: {response.status_code}")
        return None
    


In [4]:
def fetch_data(url, api_key, pool_address, start_block, end_block):
    params = {
        'module': 'account',
        'action': 'tokentx',
        'address': pool_address,
        'startblock': startblock,
        'endblock': endblock,
        'sort': 'desc',
        'apikey': api_key
    }
    
    response = requests.get(url, params=params)
    if response.status_code != 200:
        #st.error(f"API request failed with status code {response.status_code}")
        raise Exception(f"API request failed with status code {response.status_code}")
    
    data = response.json()
    if data['status'] != '1':
        #st.error(f"API returned an error: {data['result']}")
        raise Exception(f"API returned an error: {data['result']}")
    
    df = pd.DataFrame(data['result'])
    
    expected_columns = ['hash', 'blockNumber', 'timeStamp', 'from', 'to', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed', 'confirmations', 'tokenSymbol', 'value', 'tokenName']
    
    for col in expected_columns:
        if col not in df.columns:
            raise Exception(f"Expected column '{col}' is missing from the response")
    
    df.sort_values(by='timeStamp')
    
    consolidated_data = {}

    for index, row in df.iterrows():
        tx_hash = row['hash']
        
        if tx_hash not in consolidated_data:
            consolidated_data[tx_hash] = {
                'blockNumber': row['blockNumber'],
                'timeStamp': row['timeStamp'],
                'hash': tx_hash,
                'from': row['from'],
                'to': row['to'],
                'WETH_value': 0,
                'USDC_value': 0,
                'tokenName_WETH': '',
                'tokenName_USDC': '',
                'gas': row['gas'],
                'gasPrice': row['gasPrice'],
                'gasUsed': row['gasUsed'],
                'cumulativeGasUsed': row['cumulativeGasUsed'],
                'confirmations': row['confirmations']
            }
        
        if row['tokenSymbol'] == 'WETH':
            consolidated_data[tx_hash]['WETH_value'] = row['value']
            consolidated_data[tx_hash]['tokenName_WETH'] = row['tokenName']
        elif row['tokenSymbol'] == 'USDC':
            consolidated_data[tx_hash]['USDC_value'] = row['value']
            consolidated_data[tx_hash]['tokenName_USDC'] = row['tokenName']

    return pd.DataFrame.from_dict(consolidated_data, orient='index')

In [5]:
# fetch block numbers for the last hour.
# Assumes that there is at least one transaction from both pools
# and more than ten transactions total.
endblock = fetch_latest_block(ETHERSCAN_URL, ETHERSCAN_API_KEY)
startblock = endblock-5 * 60
if endblock: 
    print(f"Start Block: {startblock}")
    print(f"Stop Block: {endblock}")

try: 
    p0 = fetch_data(ETHERSCAN_URL, ETHERSCAN_API_KEY, POOL0_ADDRESS, startblock, endblock)
    endtime = int(p0['timeStamp'].iloc[0])
    beginningtime = int(p0['timeStamp'].iloc[-1])
    print(f"Successfully fetched {p0.shape[0]} swaps in the last {(endtime -beginningtime)/60:.2f} minutes.")
    
    p1 = fetch_data(ETHERSCAN_URL, ETHERSCAN_API_KEY, POOL1_ADDRESS, startblock, endblock)
    endtime = int(p1['timeStamp'].iloc[0])
    beginningtime = int(p1['timeStamp'].iloc[-1])
    print(f"Successfully fetched {p1.shape[0]} swaps in the last {(endtime -beginningtime)/60:.2f} minutes.")
except Exception as e: 
    print(e)

Start Block: 21604002
Stop Block: 21604302
Successfully fetched 210 swaps in the last 60.20 minutes.
Successfully fetched 19 swaps in the last 39.40 minutes.


In [6]:
p0.columns

Index(['blockNumber', 'timeStamp', 'hash', 'from', 'to', 'WETH_value',
       'USDC_value', 'tokenName_WETH', 'tokenName_USDC', 'gas', 'gasPrice',
       'gasUsed', 'cumulativeGasUsed', 'confirmations'],
      dtype='object')

In [7]:
p0.head()

,blockNumber,timeStamp,hash,from,to,WETH_value,USDC_value,tokenName_WETH,tokenName_USDC,gas,gasPrice,gasUsed,cumulativeGasUsed,confirmations
0x7192f1e0ebc67a2e63871ad227cb12a698eaa124f63efb4295e8ff5094840517,21604302,1736635427,0x7192f1e0ebc67a2e63871ad227cb12a698eaa124f63e...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,9952837160592063051,32742175146,Wrapped Ether,USDC,242178,6614247195,120962,1580041,1
0x778b23d28a9c6621a49a37379c160842b11da2781105f71090843c92f149e2c9,21604301,1736635415,0x778b23d28a9c6621a49a37379c160842b11da2781105...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,117057691283999443,385498311,Wrapped Ether,USDC,430374,4669421872,318424,5304326,2
0x2cbf1af88f0c11f4f84a26e24e4fbb76f70ec865ef166bbf5ceaf208621e357a,21604301,1736635415,0x2cbf1af88f0c11f4f84a26e24e4fbb76f70ec865ef16...,0x5050e08626c499411b5d0e0b5af0e83d3fd82edf,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,18609950553855617575,61232929273,Wrapped Ether,USDC,238904,2669421872,119521,119521,2
0xd7896a96347a1567f1bbc47d335f7591682f141fda71ee8fe74c7f04c3f2e1d7,21604300,1736635403,0xd7896a96347a1567f1bbc47d335f7591682f141fda71...,0x72bb58f23cf70ecbd39c7bfbad08b43e31174456,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,61517560702676178,202640000,Wrapped Ether,USDC,370084,4747940358,270480,1684607,3
0x82751e936687f0369d293b931c0b076d9c073f608a20916d4ef294bdd26469e0,21604300,1736635403,0x82751e936687f0369d293b931c0b076d9c073f608a20...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,14236620584017145469,46853086702,Wrapped Ether,USDC,242178,9330716511,121016,428105,3


Now that I have information at the transaction level.  I would like to parse specific transactions to extract the sqrtPriceX96 value which is the only value missing for our current model.  Also, assuming that I am doing this for inference, I only need transactions that fit the following criteria: (1) at least one transactions from each pool (because of the forward fill), (2) at least 10 transactions total (to ensure we can do lags and moving averages for feature extraction). 

For now, I will just truncate both to use only the last ten from each pool.  This is a shortcut for inference only that allows redunancy in the fetch for data from logs, allows for forward fill of known values, etc.

In [8]:
p0 = p0.iloc[-10:]
p1 = p1.iloc[-10:]

**Fetch sqrtPriceX96 values for the first 10 transactions in the returned list.**

In [9]:
def decode_block_data(url, swap_tx_data, block_number,transaction_hash,datatype_list,verbose=False):
    """
    
    # Call for a single block
    print(f"Block Number: {block_number}")
    swap_tx_data = pool1_tx_data[0]['data']
    block_number = int(pool1_tx_data[0]['blockNumber'],16)
    block_data = decode_block_data(swap_tx_data,block_number,verbose=True)
    
    """
    w3 = Web3(Web3.HTTPProvider(url))

    amount0, amount1, sqrtPriceX96, liquidity, tick = w3.eth.codec.decode(datatype_list,bytes.fromhex(swap_tx_data[2:]))
    return sqrtPriceX96

# Function to fetch logs from a block
def fetch_logs_for_block(url, api_key, block_number, pool_address):
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": block_number,
        "toBlock": block_number,
        "address": pool_address,
        "apikey": api_key,
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if data.get("status") == "1" and "result" in data:
        return data["result"]
    else:
        raise ValueError(f"Error fetching logs for block {block_number}: {data.get('message', 'Unknown error')}")

# Decode the sqrtPriceX96 from the logs
def decode_sqrt_price_x96(url, logs):
    sqrt_prices = []

    #import pdb
    #pdb.set_trace()
    
    # Uniswap V3 Swap event ABI
    swap_event_abi = {
        "anonymous": False,
        "inputs": [
            {"indexed": True, "internalType": "address", "name": "sender", "type": "address"},
            {"indexed": False, "internalType": "int256", "name": "amount0", "type": "int256"},
            {"indexed": False, "internalType": "int256", "name": "amount1", "type": "int256"},
            {"indexed": False, "internalType": "uint160", "name": "sqrtPriceX96", "type": "uint160"},
            {"indexed": False, "internalType": "uint128", "name": "liquidity", "type": "uint128"},
            {"indexed": False, "internalType": "int24", "name": "tick", "type": "int24"}
        ],
        "name": "Swap",
        "type": "event"
    }
    # For Decoding Data Field
    datatype_list = [i['type'] for i in swap_event_abi['inputs'] if not i['indexed']]
    raw_data = []
    for log in logs:
        if log["topics"][0] == "0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67":
            block = int(log['blockNumber'],16)
            data = log['data']
            transaction_hash=log['transactionHash']
            raw_data.append(decode_block_data(url,data,block,transaction_hash,datatype_list))
            
            # Extract sqrtPriceX96 from the data field (last 32 bytes)
            sqrt_price_x96 = int(data[-64:], 16)
            sqrt_prices.append(sqrt_price_x96)
    return raw_data



def fetch_sqrtPriceX96(block_numbers, time_stamps):
    results = {'timeStamp':[], 'blockNumber':[], 'sqrtPriceX96':[]}

    for block,timestamp in zip(block_numbers,time_stamps):
        try:
            logs = fetch_logs_for_block(ETHERSCAN_URL, ETHERSCAN_API_KEY, block, POOL0_ADDRESS)
            sqrt_prices = decode_sqrt_price_x96(ETHERSCAN_URL, logs)
            for sqrt_price in sqrt_prices:
                results['timeStamp'].append(timestamp)
                results['blockNumber'].append(block)
                results['sqrtPriceX96'].append(sqrt_price)
            print(".",end='')
                
        except Exception as e:
            print(f"Error processing block {block}: {e} (ignoring for now)")

    return results

start_time = time.time()
p0_sqrtPriceX86 = fetch_sqrtPriceX96(list(p0['blockNumber']), list(p0['timeStamp']))
p0 = p0.merge(pd.DataFrame(p0_sqrtPriceX86),on=['timeStamp','blockNumber'],how='left').dropna().reset_index(drop=True)
p1_sqrtPriceX86 = fetch_sqrtPriceX96(list(p1['blockNumber']), list(p1['timeStamp']))
p1 = p1.merge(pd.DataFrame(p1_sqrtPriceX86),on=['timeStamp','blockNumber'],how='left').dropna().reset_index(drop=True)
finish_time = time.time()
print(f"Completed fetching {len(list(p0['blockNumber']))} blocks in {finish_time-start_time}s")

...........Error processing block 21604191: Error fetching logs for block 21604191: No records found (ignoring for now)
...Error processing block 21604182: Error fetching logs for block 21604182: No records found (ignoring for now)
....Completed fetching 30 blocks in 37.69569802284241s


In [10]:
p1.head(10)

,blockNumber,timeStamp,hash,from,to,WETH_value,USDC_value,tokenName_WETH,tokenName_USDC,gas,gasPrice,gasUsed,cumulativeGasUsed,confirmations,sqrtPriceX96
0,21604212,1736634347,0x9df472b03378cce5b05cbf436c2eab236e653c307ecc...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,225395016287157758,742035788,Wrapped Ether,USDC,431056,4971835511,324590,18407762,92,1380514264484254416537537861148924
1,21604212,1736634347,0x9df472b03378cce5b05cbf436c2eab236e653c307ecc...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,225395016287157758,742035788,Wrapped Ether,USDC,431056,4971835511,324590,18407762,92,1380514291498316232896184810345703
2,21604187,1736634047,0x018beedf6c06a81b7d5f90127e5c848583e716b17e5f...,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,310000000000000000,1020595445,Wrapped Ether,USDC,436303,4206856304,227220,6397034,117,1380422656703895257410091034488680
3,21604185,1736634023,0x0861386c373a11985b564f1336df047e492d7ef9a8f7...,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,6397649033324561408,21069489319,Wrapped Ether,USDC,155645,13689339380,119727,3489120,119,1380422629693419562304873562407807
4,21604184,1736634011,0x82742936e6e80cea74edc888d8646937515cb4d8dcab...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,40000000000000000,131773946,Wrapped Ether,USDC,216886,6814960568,148785,1553374,120,1380140638775950949731639261742321
5,21604176,1736633915,0x457404d4a014abd7ff05d6a1549b62bfbe63257b8243...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,6461124066315577260,21291931972,Wrapped Ether,USDC,240078,15068312413,121018,1521632,128,1380023529804120877843275517905125
6,21604139,1736633459,0xc428fb42b6b3cbb14e837bb2221053e31549a044b4fa...,0x68d3a973e7272eb388022a5c6518d9b2a2e66fbf,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,7824044723527195218,25801215805,Wrapped Ether,USDC,240078,21902027003,121014,1095952,165,1379584029879602566561725119968368
7,21604093,1736632907,0x1136ad9a509443090df6af1729337a82ae6a086abd0c...,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,134999999808311970,446689001,Wrapped Ether,USDC,486573,4269201123,460445,2994732,211,1378112258943557522732470495737237
8,21604064,1736632559,0x4b2b9b299554eb8f6c9bb798fc5ffd88b7215f70fe27...,0x77887e7992d0a6eb7db7490330296e0f8d4136a9,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,75328222713855875,250000000,Wrapped Ether,USDC,494866,3869858592,353091,5483207,240,1378557425177325975629362336861559
9,21604064,1736632559,0x4b2b9b299554eb8f6c9bb798fc5ffd88b7215f70fe27...,0x77887e7992d0a6eb7db7490330296e0f8d4136a9,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,75328222713855875,250000000,Wrapped Ether,USDC,494866,3869858592,353091,5483207,240,1378144745772734225587348451287219


**Get price within the Pool in ETH/USDC**

To derive the price for the pool in ETH/USDC, you must use the sqrtPriceX96 value, which is the pool price immediately after the transaction takes place (including slippage).  You can see below that there is almost always a descrepency, but its not always enough to over come transaction and gas fees (see below).

In [11]:
# row to pick for the swap...used just for the example.
pool0_price_in_USDC_per_ETH  = ((p0["sqrtPriceX96"].iloc[0] / 2**96)**2 / 1e12) **-1
pool1_price_in_USDC_per_ETH  = ((p1["sqrtPriceX96"].iloc[0] / 2**96)**2 / 1e12) **-1

print(f"Pool 0 Price in USDC per ETH (at Tx: 0x...{p0['hash'].iloc[0][-4:]}): ${pool0_price_in_USDC_per_ETH:.2f}")
print(f"Pool 1 Price in USDC per ETH (at Tx: 0x...{p1['hash'].iloc[0][-4:]}): ${pool1_price_in_USDC_per_ETH:.2f}")
print(f"Difference in price: ${pool1_price_in_USDC_per_ETH-pool0_price_in_USDC_per_ETH:.2f}")

Pool 0 Price in USDC per ETH (at Tx: 0x...d6ca): $3305.92
Pool 1 Price in USDC per ETH (at Tx: 0x...2ba2): $3293.65
Difference in price: $-12.27


**Get gas fees in ETH**

Gas fees for a transaction include all the 'work' done. There is a rate of fee per unit of work (i.e. gasPrice) and then there is the work done (i.e. gasUsed).  gasPrice and gasUsed is in gwei which is 1e9 of an ETH.  so to convert to eth, each value needs to be converted with the 1e9 scaling.

In [12]:
gas_price_eth_tokens_per_unit = int(p0['gasPrice'].iloc[0])/GWEI_SCALER
gas_used_units = int(p0['gasUsed'].iloc[0]) / GWEI_SCALER
gas_fees_eth_tokens  = gas_price_eth_tokens_per_unit* gas_used_units
gas_fees_usdc_tokens = pool0_price_in_USDC_per_ETH * gas_fees_eth_tokens 
print(f"Gas Price in ETH per unit: {gas_price_eth_tokens_per_unit}")
print(f"Gas Used in GWEI units for Uniswap Transaction: {gas_used_units}")
print(f"Gas fees for this Transaction in ETH: {gas_fees_eth_tokens:.5f} (ETH)")
print(f"Gas fees for this Transaction in USDC: ${gas_fees_usdc_tokens:.2f} (USDC)")

Gas Price in ETH per unit: 44.405108188
Gas Used in GWEI units for Uniswap Transaction: 0.000121322
Gas fees for this Transaction in ETH: 0.00539 (ETH)
Gas fees for this Transaction in USDC: $17.81 (USDC)
